# DEA Coastlines generation using command line tools <img align="right" src="https://github.com/GeoscienceAustralia/dea-notebooks/raw/develop/Supplementary_data/dea_logo.jpg">

This notebook demonstrates how to run a DEA Coastlines analysis using command line tools.

### Setup

Update required packages:

In [ ]:
pip install mock

In [ ]:
pip install topojson

In [ ]:
pip install geopandas==0.10.2

In [ ]:
pip install pandas==1.3.4

Set analysis parameters:

In [ ]:
config_path = 'configs/dea_coastlines_config.yaml'
study_area = 7931
raster_version = 'testing'
vector_version = 'testing'
continental_version = 'testing'

Set working directory to top level of repo to ensure links work correctly:

In [ ]:
cd ..

### Run DEA Coastlines analysis
#### Run tidally-constrained raster generation

In [ ]:
!python dea_coastlines/raster.py --help

Example analysis:

In [ ]:
!python dea_coastlines/raster.py --config_path {config_path} --study_area {study_area} --raster_version {raster_version} --start_year 1987 --end_year 2021

#### Run vector annual shoreline and rates of change statistics generation

In [ ]:
!python dea_coastlines/vector.py --help

Example analysis:

In [ ]:
!python dea_coastlines/vector.py --config_path {config_path} --study_area {study_area} --raster_version {raster_version} --vector_version {vector_version} --baseline_year 2020

#### Run continental-scale layer generation

In [ ]:
!python dea_coastlines/continental.py --help

Example analysis:

In [ ]:
!python dea_coastlines/continental.py --vector_version {vector_version} --continental_version {continental_version} --shorelines True --ratesofchange True --hotspots True

### Clean up and remove files

In [ ]:
# rm -rf data/interim/raster/{raster_version}

In [ ]:
# rm -rf data/interim/vector/{vector_version}

In [ ]:
# rm -rf data/processed/{continental_version}